In [2]:
# import modules
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv
import re

In [3]:
# Create a dictionary of URLs for each position group

url_dict = {
    "QB": "https://www.fantasypros.com/nfl/projections/qb.php?week=draft",
    "RB": "https://www.fantasypros.com/nfl/projections/rb.php?week=draft&scoring=PPR&week=draft",
    "WR": "https://www.fantasypros.com/nfl/projections/wr.php?week=draft&scoring=PPR&week=draft",
    "TE": "https://www.fantasypros.com/nfl/projections/te.php?week=draft&scoring=PPR&week=draft",
    "FLEX": "https://www.fantasypros.com/nfl/projections/flex.php?week=draft&scoring=PPR&week=draft",
    "K": "https://www.fantasypros.com/nfl/projections/k.php?week=draft",
    "DEF": "https://www.fantasypros.com/nfl/projections/dst.php?week=draft",
}

In [4]:
# Step 2: Loop through each key-value pair in the dictionary
for name, url in url_dict.items():
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Example: Locate and extract data from a table (customize this as needed)
    table = soup.find("table")

    if table:
        headers = [header.get_text().strip() for header in table.find_all("th")]
        rows = []
        for row in table.find_all("tr"):
            cells = [cell.get_text().strip() for cell in row.find_all("td")]
            if cells:  # Only append non-empty rows
                rows.append(cells)

        # Combine headers and rows into a DataFrame
        df = pd.DataFrame(rows, columns=headers)

        # Step 3: Save each DataFrame to a separate CSV file with the variable name
        filename = os.path.join("data/scraped", f"{name}_projections.csv")
        df.to_csv(filename, index=False)

        print(f"Data from {name} ({url}) saved to {filename}")

print("Scraping and data saving completed.")

Data from QB (https://www.fantasypros.com/nfl/projections/qb.php?week=draft) saved to data/scraped/QB_projections.csv
Data from RB (https://www.fantasypros.com/nfl/projections/rb.php?week=draft&scoring=PPR&week=draft) saved to data/scraped/RB_projections.csv
Data from WR (https://www.fantasypros.com/nfl/projections/wr.php?week=draft&scoring=PPR&week=draft) saved to data/scraped/WR_projections.csv
Data from TE (https://www.fantasypros.com/nfl/projections/te.php?week=draft&scoring=PPR&week=draft) saved to data/scraped/TE_projections.csv
Data from FLEX (https://www.fantasypros.com/nfl/projections/flex.php?week=draft&scoring=PPR&week=draft) saved to data/scraped/FLEX_projections.csv
Data from K (https://www.fantasypros.com/nfl/projections/k.php?week=draft) saved to data/scraped/K_projections.csv
Data from DEF (https://www.fantasypros.com/nfl/projections/dst.php?week=draft) saved to data/scraped/DEF_projections.csv
Scraping and data saving completed.


In [33]:
# Clean the data so it can be manipulated

# Some of the positions have an extra row of column headers that need to be deleted

clean_files = [
    "QB_projections",
    "RB_projections",
    "WR_projections",
    "TE_projections",
    "FLEX_projections",
    "K_projections",
    "DEF_projections",
]

# Position mapping for each file
position_mapping = {
    "QB_projections": "QB",
    "RB_projections": "RB",
    "WR_projections": "WR",
    "TE_projections": "TE",
    "K_projections": "K",
    "DEF_projections": "DEF",
}

# List to store each cleaned dataframe in order to merge them into one final one at the end
dfs = []

for file in clean_files:
    df = pd.read_csv(f"data/scraped/{file}.csv")

    # Make sure all columns after the 'Player' column are numerical
    df.iloc[:, 2:] = (
        df.iloc[:, 2:]
        .replace({",": ""}, regex=True)
        .apply(pd.to_numeric, errors="coerce")
        .astype(float)
    )

    if file == "RB_projections":
        df = df[1:]
        df = df.rename(
            columns={
                "ATT": "RUSH_ATT",
                "YDS": "RUSH_YDS",
                "TDS": "RUSH_TDS",
                "YDS.1": "REC_YDS",
                "TDS.1": "REC_TDS",
            }
        )
    elif file == "WR_projections":
        df = df[1:]
        df = df.rename(
            columns={
                "YDS": "REC_YDS",
                "TDS": "REC_TDS",
                "YDS.1": "RUSH_YDS",
                "TDS.1": "RUSH_TDS",
            }
        )
    elif file == "TE_projections":
        df = df[1:]
        df = df.rename(columns={"YDS": "REC_YDS", "TDS": "REC_TDS"})

    elif file == "QB_projections":
        df = df[1:]
        df = df.rename(
            columns={
                "YDS": "PASS_YDS",
                "TDS": "PASS_TDS",
                "ATT.1": "RUSH_ATT",
                "YDS.1": "RUSH_YDS",
                "TDS.1": "RUSH_TDS",
            }
        )

    elif file == "FLEX_projections":
        df = df[1:]
        df = df.rename(
            columns={
                "YDS": "RUSH_YDS",
                "TDS": "RUSH_TDS",
                "YDS.1": "REC_YDS",
                "TDS.1": "REC_TDS",
            }
        )
        df["POS"] = df["POS"].astype(str).apply(lambda x: re.sub(r"[^a-zA-Z]", "", x))

        # Continue to the next file without appending FLEX to dfs
        continue

    # Assign position based on the filename using position_mapping
    pos = position_mapping[file]
    df["POS"] = pos

    # Append the cleaned DataFrame to the list
    dfs.append(df)

    # Save cleaned files to the clean subdirectory
    filename = os.path.join("data/clean", f"{file}_clean.csv")
    df.to_csv(filename, index=False)

    print(f"Data from data/scraped/{file}.csv cleaned and saved to {filename}")

# Concatenate all DataFrames in the list into one
final_df = pd.concat(dfs, ignore_index=True)

# Move 'POS' column to be the second column
columns = final_df.columns.tolist()
if "POS" in columns:
    columns.insert(1, columns.pop(columns.index("POS")))
    final_df = final_df[columns]


# Optionally, save the final merged DataFrame to a CSV file
final_filename = os.path.join("data/clean", "all_projections_clean.csv")
final_df.to_csv(final_filename, index=False)

print(f"All data merged and saved to {final_filename}")

print("Data cleaning completed.")

Data from data/scraped/QB_projections.csv cleaned and saved to data/clean/QB_projections_clean.csv
Data from data/scraped/RB_projections.csv cleaned and saved to data/clean/RB_projections_clean.csv
Data from data/scraped/WR_projections.csv cleaned and saved to data/clean/WR_projections_clean.csv
Data from data/scraped/TE_projections.csv cleaned and saved to data/clean/TE_projections_clean.csv
Data from data/scraped/K_projections.csv cleaned and saved to data/clean/K_projections_clean.csv
Data from data/scraped/DEF_projections.csv cleaned and saved to data/clean/DEF_projections_clean.csv
All data merged and saved to data/clean/all_projections_clean.csv
Data cleaning completed.


In [34]:
df = pd.read_csv("data/clean/all_projections_clean.csv")
print(df.dtypes)
df.head()

Player       object
POS          object
ATT         float64
CMP         float64
PASS_YDS    float64
PASS_TDS    float64
INTS        float64
RUSH_ATT    float64
RUSH_YDS    float64
RUSH_TDS    float64
FL          float64
FPTS        float64
REC         float64
REC_YDS     float64
REC_TDS     float64
FG          float64
FGA         float64
XPT         float64
SACK        float64
INT         float64
FR          float64
FF          float64
TD          float64
SAFETY      float64
PA          float64
YDS AGN     float64
dtype: object


,Player,POS,ATT,CMP,PASS_YDS,PASS_TDS,INTS,RUSH_ATT,RUSH_YDS,RUSH_TDS,...,FGA,XPT,SACK,INT,FR,FF,TD,SAFETY,PA,YDS AGN
0,Jalen Hurts PHI,QB,520.2,339.7,3810.4,24.4,12.1,145.3,624.2,11.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Josh Allen BUF,QB,564.7,368.0,4060.0,27.8,14.0,108.2,556.7,8.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Lamar Jackson BAL,QB,463.6,303.7,3567.1,25.0,10.6,146.1,836.1,5.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Patrick Mahomes II KC,QB,584.8,392.2,4371.2,32.7,11.2,72.0,376.4,1.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anthony Richardson IND,QB,511.7,325.6,3503.9,20.0,11.4,127.6,646.0,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df[df["Player"] == "Justin Jefferson MIN"]["REC_YDS"]

259    1441.3
Name: REC_YDS, dtype: float64

In [7]:
# # URLs to scrape from

# QB = "https://www.fantasypros.com/nfl/projections/qb.php?week=draft"
# RB = "https://www.fantasypros.com/nfl/projections/rb.php?week=draft&scoring=PPR&week=draft"
# WR = "https://www.fantasypros.com/nfl/projections/wr.php?week=draft&scoring=PPR&week=draft"
# TE = "https://www.fantasypros.com/nfl/projections/te.php?week=draft&scoring=PPR&week=draft"
# FLEX = "https://www.fantasypros.com/nfl/projections/flex.php?week=draft&scoring=PPR&week=draft"
# K = "https://www.fantasypros.com/nfl/projections/k.php?week=draft"
# DEF = "https://www.fantasypros.com/nfl/projections/dst.php?week=draft"

# urls = [QB, RB, WR, TE, FLEX, K, DEF]

In [8]:
# # Make a get request
# r = requests.get(url)

# # Initialize soup object
# soup = BeautifulSoup(r.text, "html.parser")

In [9]:
# # Find the table
# table = soup.find("table")

In [10]:
# # print(table)

# # Extract table data

# # Extract headers
# headers = []
# for header in table.find_all("th"):
#     headers.append(header.get_text().strip())


# # Extract rows
# rows = []
# for row in table.find_all("tr"):
#     cells = row.find_all("td")
#     cells = [cell.get_text().strip() for cell in cells]
#     if cells:
#         rows.append(cells)

In [11]:
# # Check if we can see the data

# for header in headers:
#     print(header, end=" | ")
# # print('\n' + '-'*40)

# for row in rows:
#     print(" | ".join(row))

In [12]:
# print(headers)
# print(rows[1:])

In [13]:
# # Save data as csv

# rows = rows[1:]

# with open("rb_projections", "w") as file:
#     writer = csv.writer(file)
#     writer.writerow(headers)
#     for row in rows:
#         writer.writerow(row)

In [14]:
# data = pd.read_csv("rb_projections")

# data.head(25)

# # data.dtypes

# data["YDS"] = data["YDS"].str.replace(",", "")
# data["YDS"] = pd.to_numeric(data["YDS"], errors="coerce")
# data["YDS"] = data["YDS"].astype("float64")

# data.head()

In [15]:
# # Overwrite the csv file with the new cleaned data

# data.to_csv("rb_projections", index=False)